Following some of the examples here: https://github.com/pytorch/fairseq/tree/master/examples/roberta

In [1]:
from fairseq.models.roberta import RobertaModel

In [2]:
roberta = RobertaModel.from_pretrained('/projects/deepgreen/pstjohn/roberta_base_checkpoint/',
                                       checkpoint_file='checkpoint_best.pt')
_ = roberta.eval()  # disable dropout (or leave in train mode to finetune)

In [3]:
# https://www.uniprot.org/uniprot/P14618.fasta
example_sequence = \
"""
MSKPHSEAGTAFIQTQQLHAAMADTFLEHMCRLDIDSPPITARNTGIICTIGPASRSVET
LKEMIKSGMNVARLNFSHGTHEYHAETIKNVRTATESFASDPILYRPVAVALDTKGPEIR
TGLIKGSGTAEVELKKGATLKITLDNAYMEKCDENILWLDYKNICKVVEVGSKIYVDDGL
ISLQVKQKGADFLVTEVENGGSLGSKKGVNLPGAAVDLPAVSEKDIQDLKFGVEQDVDMV
FASFIRKASDVHEVRKVLGEKGKNIKIISKIENHEGVRRFDEILEASDGIMVARGDLGIE
IPAEKVFLAQKMMIGRCNRAGKPVICATQMLESMIKKPRPTRAEGSDVANAVLDGADCIM
LSGETAKGDYPLEAVRMQHLIAREAEAAIYHLQLFEELRRLAPITSDPTEATAVGAVEAS
FKCCSGAIIVLTKSGRSAHQVARYRPRAPIIAVTRNPQTARQAHLYRGIFPVLCKDPVQE
AWAEDVDLRVNFAMNVGKARGFFKKGDVVIVLTGWRPGSGFTNTMRVVPVP
"""

Unlike the roberta example on fairseq, we're not using the GPT-2 byte-pair encoder, so the standard `roberta.encode` and `roberta.decode` methods won't work

In [4]:
def encode(sequence):
    input_sequence = ' '.join(sequence.replace('\n', ''))
    return roberta.task.source_dictionary.encode_line(input_sequence)

def decode(tokens):
    return roberta.task.source_dictionary.string(tokens).replace(' ', '')

In [5]:
tokens = encode(example_sequence)
tokens

tensor([20,  8, 15, 14, 21,  8,  9,  5,  6, 11,  5, 17, 12, 16, 11, 16, 16,  4,
        21,  5,  5, 20,  5, 13, 11, 17,  4,  9, 21, 20, 23, 10,  4, 13, 12, 13,
         8, 14, 14, 12, 11,  5, 10, 18, 11,  6, 12, 12, 23, 11, 12,  6, 14,  5,
         8, 10,  8,  7,  9, 11,  4, 15,  9, 20, 12, 15,  8,  6, 20, 18,  7,  5,
        10,  4, 18, 17,  8, 21,  6, 11, 21,  9, 19, 21,  5,  9, 11, 12, 15, 18,
         7, 10, 11,  5, 11,  9,  8, 17,  5,  8, 13, 14, 12,  4, 19, 10, 14,  7,
         5,  7,  5,  4, 13, 11, 15,  6, 14,  9, 12, 10, 11,  6,  4, 12, 15,  6,
         8,  6, 11,  5,  9,  7,  9,  4, 15, 15,  6,  5, 11,  4, 15, 12, 11,  4,
        13, 18,  5, 19, 20,  9, 15, 23, 13,  9, 18, 12,  4, 22,  4, 13, 19, 15,
        18, 12, 23, 15,  7,  7,  9,  7,  6,  8, 15, 12, 19,  7, 13, 13,  6,  4,
        12,  8,  4, 16,  7, 15, 16, 15,  6,  5, 13, 17,  4,  7, 11,  9,  7,  9,
        18,  6,  6,  8,  4,  6,  8, 15, 15,  6,  7, 18,  4, 14,  6,  5,  5,  7,
        13,  4, 14,  5,  7,  8,  9, 15, 

In [6]:
from textwrap import wrap
decoded_sequence = '\n'.join(wrap(decode(tokens), width=60))
print(decoded_sequence)

MSKPHSEAGTAFIQTQQLHAAMADTFLEHMCRLDIDSPPITARNTGIICTIGPASRSVET
LKEMIKSGMNVARLNFSHGTHEYHAETIKNVRTATESFASDPILYRPVAVALDTKGPEIR
TGLIKGSGTAEVELKKGATLKITLDNAYMEKCDENILWLDYKNICKVVEVGSKIYVDDGL
ISLQVKQKGADFLVTEVENGGSLGSKKGVNLPGAAVDLPAVSEKDIQDLKFGVEQDVDMV
FASFIRKASDVHEVRKVLGEKGKNIKIISKIENHEGVRRFDEILEASDGIMVARGDLGIE
IPAEKVFLAQKMMIGRCNRAGKPVICATQMLESMIKKPRPTRAEGSDVANAVLDGADCIM
LSGETAKGDYPLEAVRMQHLIAREAEAAIYHLQLFEELRRLAPITSDPTEATAVGAVEAS
FKCCSGAIIVLTKSGRSAHQVARYRPRAPIIAVTRNPQTARQAHLYRGIFPVLCKDPVQE
AWAEDVDLRVNFAMNVGKARGFFKKGDVVIVLTGWRPGSGFTNTMRVVPVP


In [7]:
decoded_sequence == example_sequence[1:-1]  # stripping leading and tailing newlines

True

In [10]:
import torch
if torch.cuda.is_available():
    print("Using the GPU")
    roberta.cuda()

with torch.no_grad():
    features = roberta.extract_features(tokens.to(torch.int64))

Using the GPU


In [11]:
features

tensor([[[ 0.1737,  0.0976, -0.0376,  ...,  0.0724, -0.0361,  0.0857],
         [-0.1052,  0.2354, -0.1681,  ..., -0.0185,  0.1915, -0.1655],
         [-0.0129,  0.0157, -0.1035,  ..., -0.0133,  0.1601, -0.0342],
         ...,
         [ 0.0780,  0.2138, -0.0877,  ..., -0.0448,  0.2261,  0.1889],
         [ 0.1001,  0.2172,  0.1093,  ...,  0.0376, -0.0170, -0.0922],
         [ 0.1199,  0.2842,  0.0715,  ...,  0.0889, -0.0753,  0.1925]]],
       device='cuda:0')